In [ ]:
MAP_KEY = '5e8bad8d50fa1ca84ea72175e2bace34'# change this key to user key

# check how many transactions we have
import pandas as pd
url = 'https://firms.modaps.eosdis.nasa.gov/mapserver/mapkey_status/?MAP_KEY=' + MAP_KEY
try:
  df = pd.read_json(url,  typ='series')
  display(df)
except:
  # possible error, wrong MAP_KEY value, check for extra quotes, missing letters
  print ("There is an issue with the query. \nTry in your browser: %s" % url)


In [ ]:
# let's create a simple function that tells us how many transactions we have used.
# We will use this in later examples

def get_transaction_count() :
  count = 0
  try:
    df = pd.read_json(url,  typ='series')
    count = df['current_transactions']
  except:
    print ("Error in our call.")
  return count

tcount = get_transaction_count()
print ('Our current transaction count is %i' % tcount)

---

## API/data_availability

---

In [ ]:
da_url = 'https://firms.modaps.eosdis.nasa.gov/api/data_availability/csv/' + MAP_KEY + '/all'
df = pd.read_csv(da_url)
display(df)

**data_id** column shows the dataset id which we will need in later queries:
- 'NRT' means this is Near Real-Time dataset but it may also includes Real Time (RT) and Ultra Real Time (URT) data [click here more info on URT/RT](https://www.earthdata.nasa.gov/data/tools/firms/faq)
- 'SP' or Standard Processing; standard data products are an internally consistent, well-calibrated record of the Earth’s geophysical properties to support science. There is a multi-month lag in this dataset availability. [more information on SP vs NRT](https://www.earthdata.nasa.gov/data/tools/firms/faq)
- BA_MODIS is for MODIS burned areas product

**min_date** and **max_date** columns provide the available date range for these datasets. Dates are based on GMT

In [4]:
# now let's see how many transactions we use by querying this end point

start_count = get_transaction_count()
pd.read_csv(da_url)
end_count = get_transaction_count()
print ('We used %i transactions.' % (end_count-start_count))

# now remember, after 10 minutes this will reset

We used 5 transactions.


---

## API/area

---

Fire detection hotspots based on area, date and sensor. For more information visit https://firms.modaps.eosdis.nasa.gov/api/area

The end point expects these parameters: [MAP_KEY], [SOURCE], [AREA_COORDINATES],[DAY_RANGE] and optionally [DATE] for historical data

**NOTE** - querying the entire world for VIIRS can return between 30,000 - 100,000+ records per day


In [ ]:
# in this example let's look at VIIRS NOAA-20, entire world and the most recent day
area_url = 'https://firms.modaps.eosdis.nasa.gov/api/area/csv/' + MAP_KEY + '/VIIRS_NOAA20_NRT/world/1'
start_count = get_transaction_count()
df_area = pd.read_csv(area_url)
end_count = get_transaction_count()
print ('We used %i transactions.' % (end_count-start_count))

df_area

In [ ]:
# We can also focus on a smaller area ex. South Asia and get the last 3 days of records
area_url = 'https://firms.modaps.eosdis.nasa.gov/api/area/csv/' + MAP_KEY + '/VIIRS_NOAA20_NRT/54,5.5,102,40/3'
df_area = pd.read_csv(area_url)
df_area

List of supported countries and their 3-letter codes. This may be easier to read in html format https://firms.modaps.eosdis.nasa.gov/api/countries/?format=html however, you won't be able to see the exent box defined for each country.

Example below shows how you can view from Python.

In [ ]:
# We can also focus on smaller area ex. South Asia and get last 3 days of records
countries_url = 'https://firms.modaps.eosdis.nasa.gov/api/countries'
df_countries = pd.read_csv(countries_url, sep=';')
df_countries

---

## API/country

---

In [ ]:
# Let's see last four days MODIS data for Thailand
thai_url = 'https://firms.modaps.eosdis.nasa.gov/api/country/csv/' + MAP_KEY + '/MODIS_NRT/THA/5'
df_thai = pd.read_csv(thai_url)
df_thai

## Change data format "acq_time"

In [ ]:
import datetime

# Ensure "acq_time" is a string, then pad with zeros
df_thai["acq_time"] = df_thai["acq_time"].astype(str).str.zfill(4)

# Convert to HH:MM format
df_thai["acq_time"] = pd.to_datetime(df_thai["acq_time"], format="%H%M").dt.time
# df_thai["acq_time"] = df_thai["acq_time"].apply(lambda x: datetime.datetime.strptime(x, "%H:%M:%S").time())

# Check the result
print(df_thai.dtypes)
print(df_thai["acq_time"].head())  # Display first few converted times

In [ ]:
unique_values = df_thai["acq_time"].unique()
print(unique_values)

In [ ]:
df_thai.head(100)

## Hive partition

In [12]:
import numpy as np
import pandas as pd
import pyarrow

In [13]:
# Extract components from the "acq_date" and "acq_time" columns
df_thai["acq_year"] = pd.to_datetime(df_thai["acq_date"]).dt.year
df_thai["acq_month"] = pd.to_datetime(df_thai["acq_date"]).dt.month
df_thai["acq_day"] = pd.to_datetime(df_thai["acq_date"]).dt.day
df_thai["acq_hour"] = df_thai["acq_time"].apply(lambda x: x.hour)
df_thai["acq_minute"] = df_thai["acq_time"].apply(lambda x: x.minute)

# Write DataFrame to a directory "output_parquet" partitioned by retrieval_time
df_thai.to_parquet(
    "df_thai",
    partition_cols=["acq_year", "acq_month", "acq_day", "acq_hour", "acq_minute"],
    engine="pyarrow",
    index=False
)